# Technical Report: Improving domain adaptation with causal graphs
2023 Spring

Student: Yufeng Wu

Advisor: Prof. Rohit Bhattacharya

## I. Proposed Research Contribution

We propose a new method that utilizes information derived from causal Directed Acyclic Graphs (DAGs) to identify the stable components of a predictive model when faced with distribution shifts. This method enables us to selectively retrain only the necessary parts of the model using data from the altered environment. Our primary objective is to outperform the baseline model in the shifted environment, surpassing the results that would be achieved by training a completely new model from scratch using the limited data available from the shifted environment.


## II. Problem Set Up

The goal of this research is to propose a novel method of achieving domain adaptation when facing distribution shift between the source and the target environment.  

Suppose we are given a causal DAG which involves Y (the target variable), V (a set of predictors of Y, all of which are causal ancesters of Y), and possibly U (a set of unobserved variables in the DAG). We further assume that the DAG factorization across the two environments are the same, but some of the terms of the factorization may be different.

We further assume that the information of which edge has shifted is given to us. That is, we do not consider how to detect distribution shift in this research. 

We have a large amount of training data available to us in the training environment ($N$). In out problem, we consider the case where we have a few data points ($M$) in the shifted environment, where $N >> M$. 

The task is to train a predictive model that achieves the highest accuracy in the shifted environment.

## III. Background

#### Definition of stability and distribution shift (Subbaswamy et al. in "unifying framework"):
We graphically charaterize instability in terms of edges in the graph of the data generating process. Assume that there is a set of environments such that a predictive problem is mapped to the same graph structure $G$. However, each environment is a different instantiation of that graph such that certain mechanisms differ. Thus, the factorization of the data distribution is the same in each environment, but terms in the factorization corresponding to shifts will vary across environments. 


#### Defintion of unstable edge (modified from Subbaswamy et al. in "unifying framework"):
An edge $X \to Y$ is said to be unstable if, in 2 different environments of $G$, the distribution $P(Y(x', V(x)) - Y(x))$ changes, where:

- $V = pa(Y) \setminus X$

- $Y(x)$ is the counterfactual value of $Y$ had $X$ been $x$

- $Y(x', V(x))$ is the counterfactual value of $Y$ had $X$ been $x'$ and had $V$ been counterfactually generated under $X = x$.

#### Using unstable edge to model common types of distribution shifts: 

The above definition is able to model:

1. Edge-strength shift: An edge strength shift in edge $X \to Y$ corresponds to a change in the natural direct effect: for $V = pa(Y) \setminus X$, $E(Y(x', V(x)) - Y(x))$ changes. 

Notice that when this expected value changes, the distribution $P(Y(x', V(x)) - Y(x))$ necessarily changes too. Thus, our definition of unstable edge is able to capture edge-strength shift.

2. Mechanism shift: A shift in the mechanism generating a variable $Y$ corresponds to arbitrary changes in the distribution $P(Y∣pa(Y))$. Suppose $Y = f(pa(Y)) + \epsilon$ is the deterministic data generating process of $Y$ given its parents, where $f$ is a function representing the relationship between $Y$ and its parents $pa(Y)$, and $\epsilon$ is an error term that is independent of $pa(Y)$.

Now, let's consider a mechanism shift in the edges $pa(Y) \to Y$. If the function $f$ changes to $f'$, the data-generating process becomes $Y = f'(pa(Y)) + \epsilon$. This change in the mechanism affects the distribution $P(Y∣pa(Y))$. 

Suppose $X \in pa(Y)$ and $V = pa(Y) \setminus X$. Let's examine the edge $X\to Y$ first. Under the counterfactual scenarios $Y(x)$ and $Y(x', V(x))$, the values of $Y$ are determined by the respective functions $f$ and $f'$ before and after the mechanism shift, which means the difference in counterfactual values, $Y(x', V(x)) - Y(x)$, will be influenced by this mechanism shift.

Since the mechanism shift can lead to arbitrary changes in the distribution $P(Y∣pa(Y))$, it can also cause changes in the distribution $P(Y(x', V(x)) - Y(x))$. Thus, our definition of unstable edge is able to capture the case when mechanism shift happens. 

In fact, when there are arbitrary changes in the distribution $P(Y∣pa(Y))$, it is likely that, for each $X_i \in pa(Y)$ and $V_i = pa(Y) \setminus X_i$, the distributions $P(Y(x_i', V_i(x_i))) - Y(x_i))$ changes. Thus, we should mark all edges coming into $Y$ as unstable.

#### Generalized Additive Model

A Generalized Additive Model (GAM) takes the form

$ g(\operatorname {E}(Y))=\beta _{0}+f_{1}(x_{1})+f_{2}(x_{2})+\cdots +f_{m}(x_{m}) $

where $f_i$ can be arbitrarily complex functions and $x_i$'s are the predictors of $Y$. 

The link function $g(\cdot)$ serves the following purposes:
1. it allows for the modeling of relationships between the predictor variables and the response variable that may not be linear, by transforming the expected value of the response variable.
2. it ensures that the expected value of the response variable lies within the permissible range, especially for response variables with bounded ranges (e.g., probabilities must be between 0 and 1).

Some common link functions used in GAMs include:

- Identity link: $g(E(Y)) = E(Y)$, used for continuous response variables in linear regression models.
- Logit link: $g(E(Y)) = \log\left(\frac{E(Y)}{1 - E(Y)}\right)$, used for binary response variables in logistic regression models.
- Log link: $g(E(Y)) = \log(E(Y))$, used for count or rate data in Poisson and negative binomial regression models.

The choice of link function depends on the distribution of the response variable and the desired relationship between the predictor variables and the response variable.


## IV. Finding stable component in a GAM using d-seperation rules

Suppose we are given a set of observed causal ancestors of $Y$, the causal relationship between them and possibly some unobserved variables, and which edges in the DAG are unstable across the source and target environments. Now, if $Y$ can be modeled using a GAM, how can we determine which variables are stable across the two environments?

#### Definition of a stable predictor

A predictor is stable if the association between such predictor and $Y$ are constant across two environments, given all the other variables that are also in the model. In other words, a predictor is stable if the portion of its effect to $Y$ that does not go through any other predictors is constant. 

#### Definition of a stable component

A stable component of a GAM across the source and target environment is a collection/set of stable predictors. 

#### Finding the stable component

The following algorithm outputs the set of predictors that forms a stable component of a GAM. 

Step 1: draw a square around all the predictors in the model to indicate that, when training a predictive model on $Y$ using the predictors $X_1, X_2, ..., X_n$, it is analogous to conditioning on them in the DAG. This is because, in the context of a DAG, conditioning on variables means considering their values in the analysis, which is what the machine learning model does when using them as predictors.

Step 2: for each predictor in the ML model, check all the paths from that variable to Y, including both front-door and back-door paths. A predictor is stable if there is no unstable path from that predictor to $Y$, when conditioning on the rest of the predictors. 

The set of all the stable predictors found in step 2 is the stable component that is directly transferable from the source to the target environment.

#### Example

Suppose we are given the following data generating process, where $U1$ and $U2$ are unobserved and unstable edges across the two environments are colored red. 

<img src="example_1_DAG.png" alt="pic" width="30%">

To determine a stable component of a GAM that predicts $Y$ using $X_1, X_2, X_3$, we need to draw a square around all the predictors, see below. By d-separation, when learning a ML model that predicts $Y$ using $X_1, X_2, X_3$, the association between $X_1$ and $Y$ is learned through the path $X_1 \to U_2 \to Y$ only, because the other flow of association from $X_1$ to $Y$ that goes through $X_2$ is blocked since $X_2$ is being "conditioned on" during the model training process. Note: this is because -- when learning the association between $X_1$ and $Y$, we're essentially asking the question: how does $Y$ change depending on $X_1$, given all else constant. The "given all else constant" part is equivalent to "conditioning" in causal terms, because we are fixing $X_2$ and $X_3$ to some specific values when learning the relationship between $X_1$ and $Y$. 

<img src="example_1_conditioned.png" alt="pic" width="30%">

Therefore, if we learned a model $g(\operatorname {E}(Y))=\beta _{0}+f_{1}(X_{1})+f_{2}(X_{2}) +f_{3}(X_{3}) $ that is able to predict $Y$ in the source environment, then $f_1(X_1)$ and $f_3(X_3)$ are stable predictors across the two environments whereas $f_2(X_2)$ is not stable because there is an unstable path between $X_2$ and $Y$. So, the association learned in the source environment $f_2(X_2)$ is not transportable.
